In [ ]:
%pip install --upgrade --quiet langchain_community langchain-openai langchainhub 
%pip install --upgrade --quiet  azure-search-documents>=11.4
%pip install --upgrade --quiet  azure-identity
%pip install python-dotenv

In [ ]:
import os
import dotenv

dotenv.load_dotenv()

ai_serch_svc = os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")
ai_search_index = os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
ai_search_api_key = os.getenv("AZURE_AI_SEARCH_API_KEY")
phi_key = os.getenv("PHI_SERVERLESS_KEY")
llama_key = os.getenv("LLAMA_MANAGED_KEY")

Sample Question:

1. I’m an ICU nurse looking to move to Nanaimo, do you have any openings?
* Follow up question: what’s it like living in Nanaimo?
* Follow up question: what’re house prices like?

2. Show average house price in Nanaimo from… somewhere? It must be public 
* Follow up question: what would I need to qualify for one of those roles?

#### Create Azure AI Search Retriever

In [ ]:
from langchain_community.retrievers import AzureAISearchRetriever

retriever = AzureAISearchRetriever(
    api_key=ai_search_api_key,
    index_name=ai_search_index,
    service_name=ai_serch_svc,
    top_k=5,
    content_key="chunk"
)

#### Prompt

In [ ]:
from langchain import hub
prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [ ]:
q1 = "I am an ICU nurse looking to move to Nanaimo, do you have any openings?"
q2 = "Do I qualify for this role if I have 5 years of experience as a critical care nurse?"
q3 = "How is it like living in that city?"

#### GPT 4-o

In [ ]:
from langchain_openai import AzureChatOpenAI

gpt4o = AzureChatOpenAI(
    api_version="2024-06-01",
    azure_deployment="gpt-4o",
    temperature=0,
    top_p=0.95,
    max_tokens=1000,
)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage

gpt4o_retrieval_chain = create_retrieval_chain(retriever, create_stuff_documents_chain(gpt4o, prompt))

In [ ]:
chat_history = []

gptr1 = gpt4o_retrieval_chain.invoke({"input": q1, "chat_history": chat_history})
print("\n" + gptr1["answer"])
chat_history.extend([HumanMessage(content=q1), gptr1["answer"]])

In [ ]:
gptr2 = gpt4o_retrieval_chain.invoke({"input": q2, "chat_history": chat_history})
print("\n" + gptr2["answer"])
chat_history.extend([HumanMessage(content=q2), gptr2["answer"]])

In [ ]:
print("\n" + gpt4o_retrieval_chain.invoke({"input": q3, "chat_history": chat_history})["answer"])
chat_history.clear()

#### Phi - Managed endpoint

- Phi 3 medium 128K
The allowed skus for this model are 
1. Standard_NC48ads_A100_v4 - $6616.59 per month / 3 year reserved
2. Standard_NC96ads_A100_v4
3. Standard_ND96amsr_A100_v4

- Phi 3 small 128K
The allowed skus for this model are 
1. Standard_NC24ads_A100_v4 - $3308.29 per month / 3 year reserved
2. Standard_NC48ads_A100_v4
3. Standard_NC96ads_A100_v4
4. Standard_ND96amsr_A100_v4

In [ ]:
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLChatOnlineEndpoint,
    AzureMLEndpointApiType,
    CustomOpenAIChatContentFormatter,
)

phi3s128k_model = AzureMLChatOnlineEndpoint(
    endpoint_url="https://Phi-3-small-128k-instruct-swlss.eastus2.models.ai.azure.com/v1/chat/completions",
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key=phi_key,
    content_formatter=CustomOpenAIChatContentFormatter(),
    model_kwargs={"temperature": 0, "top_p": 0.95, "max_new_tokens": 1000}
)

In [ ]:
phi3s128k_retrieval_chain = create_retrieval_chain(retriever, create_stuff_documents_chain(phi3s128k_model, prompt))

In [ ]:
phir1 = phi3s128k_retrieval_chain.invoke({"input": q1, "chat_history": chat_history})
print("\n" + phir1["answer"])
chat_history.extend([HumanMessage(content=q1), phir1["answer"]])

In [ ]:
phir2 = phi3s128k_retrieval_chain.invoke({"input": q2, "chat_history": chat_history})
print("\n" + phir2["answer"])
chat_history.extend([HumanMessage(content=q2), phir2["answer"]])

In [ ]:
print("\n" + phi3s128k_retrieval_chain.invoke({"input": q3, "chat_history": chat_history})["answer"])
chat_history.clear()

#### LLama - Managed endpoint

- LLama 3.1 70B

The allowed skus for this model are
1. Standard_NC96ads_A100_v4
2. Standard_ND96asr_v4 
3. Standard_ND96amsr_A100_v4

- LLama 3.1 8B

The allowed skus for this model are 
1. Standard_ND40rs_v2
2. Standard_NC24ads_A100_v4
3. Standard_NC48ads_A100_v4
4. Standard_NC96ads_A100_v4
5. Standard_ND96amsr_A100_v4

In [ ]:
llama318k_model = AzureMLChatOnlineEndpoint(
    endpoint_url="https://js-aistudio-proj-eus2-ooqvl.eastus2.inference.ml.azure.com/score",
    endpoint_api_type=AzureMLEndpointApiType.dedicated,
    endpoint_api_key=llama_key,
    content_formatter=CustomOpenAIChatContentFormatter(),
    model_kwargs={"temperature": 0, "top_p": 0.95, "max_new_tokens": 1000}
)

In [ ]:
llama_retrieval_chain = create_retrieval_chain(retriever, create_stuff_documents_chain(llama318k_model, prompt))

In [ ]:
llamar1 = llama_retrieval_chain.invoke({"input": q1, "chat_history": chat_history})
print("\n" + llamar1["answer"])
chat_history.extend([HumanMessage(content=q1), llamar1["answer"]])

In [ ]:
llamar2 = llama_retrieval_chain.invoke({"input": q2, "chat_history": chat_history})
print("\n" + llamar2["answer"])
chat_history.extend([HumanMessage(content=q2), llamar2["answer"]])

In [ ]:
print("\n" + llama_retrieval_chain.invoke({"input": q3, "chat_history": chat_history})["answer"])
chat_history.clear()

#### Responses

Sr. No. | Question | GPT 4-o | Phi - 3 small | LLama - 3.1
--- | --- | --- | --- | ---
1 | I am an ICU nurse looking to move to Nanaimo, do you have any openings? | Yes, there are current and future opportunities for Registered Nurses, including ICU nurses, at Nanaimo Regional General Hospital (NRGH). The job postings indicate that Island Health is looking for patient-focused team players to join their team of compassionate and dedicated nurses who provide care to acutely ill patients in their Critical Care units. If you are interested, you should consider applying for the positions listed under the Expression of Interest (EOI) numbers JD4916 and JD5029. These positions are for Registered Nurses in various departments, including the Intensive Care Unit (ICU) and High Acuity Unit (HAU). To qualify, you need to have: - Registration with the BC College of Nurses and Midwives as a practicing RN registrant. - Minimum 2 years of recent Critical Care nursing experience and/or an accredited Critical Care course. - Demonstrated ability in arrhythmia interpretation as evidenced by a mark of 75% on the VIHA ECG Challenge exam. - Advanced Cardiac Life Support and CRRT (continuous renal replacement therapy) are considered assets. Relocation assistance may also be available to qualified applicants. If you meet these qualifications and are interested in the position, you should apply to continue the conversation with Island Health. | Yes, we currently have an opening for a Registered Nurse in the ICU & High Acuity Unit at Nanaimo Regional General Hospital. This position is a Level 3 GRID/PAY LEVEL role, as per the Nurses Bargaining Association HSCIS NO.: 21001, and is represented by the BCNU union. The job involves providing direct patient care in the Intensive Care Unit (ICU) and High Acuity Unit (HAU), with a focus on acutely ill patients. The staffing ratio is 1:1 for ventilated and critically unstable patients. The ICU nurses are part of the Code Blue team, except for the Cowichan District and West Coast General Hospitals. Continuous Renal Replacement Therapy (CRRT) and organ donor maintenance are available at the Victoria and Nanaimo Hospitals. The High Acuity units provide care for adult patients requiring increased monitoring and resuscitation in a multidisciplinary environment. The nurse to patient ratio in this unit is 1:2. The position requires a minimum of 2 years recent Critical Care nursing experience and or an accredited Critical Care course. Advanced Cardiac Life Support and CRRT (continuous renal replacement therapy) are assets. If you are interested, please apply through our job portal. | According to the job descriptions provided, there are openings for ICU/HAU Critical Care Nurses in Nanaimo, specifically at Nanaimo Regional General Hospital. The job description (JD3027) mentions that the position is available at Nanaimo Regional General Hospital, which is one of the facilities listed. However, I would recommend checking the Island Health website or contacting the Human Resources department directly to confirm the availability of positions and to inquire about the application process. They may also have more information about the specific requirements and qualifications for the position. Additionally, you can also check the Island Health website for other ICU nurse positions that may be available in Nanaimo or other locations on Vancouver Island. |
2 | Do I qualify for this role if I have 5 years of experience as a critical care nurse? | Based on the job description for the ICU/HAU Critical Care Nurse position at Nanaimo Regional General Hospital, you would likely qualify for the role if you have 5 years of experience as a critical care nurse. The qualifications for this position include: - Registration with the BC College of Nurses and Midwives as a practicing RN registrant. - Minimum 2 years of recent Critical Care nursing experience and/or an accredited Critical Care course. - Demonstrated ability in arrhythmia interpretation as evidenced by a mark of 75% on the VIHA ECG Challenge exam. - Advanced Cardiac Life Support (ACLS) and CRRT (continuous renal replacement therapy) are considered assets. Since you have 5 years of experience in critical care, you meet the experience requirement. If you also meet the other qualifications, such as registration with the BC College of Nurses and Midwives and the ability to interpret arrhythmias, you should be well-qualified for the role. Advanced Cardiac Life Support and CRRT are assets, so having these certifications would further strengthen your application. If you meet these criteria, you should consider applying for the position. | Yes, with 5 years of experience as a critical care nurse, you would be well-qualified for this role. The job description specifies a minimum of 2 years recent Critical Care nursing experience, and your 5 years of experience would exceed this requirement. Additionally, the job description mentions that Advanced Cardiac Life Support and CRRT (continuous renal replacement therapy) are assets, which suggests that these skills would be beneficial but not necessarily required. However, it's always a good idea to check the specific requirements of the job posting to ensure you meet all the qualifications. | According to the job description (JD3027), the minimum requirement for the ICU/HAU Critical Care Nurse position is 2 years of recent, related experience, or an equivalent combination of education, training, and experience. Since you have 5 years of experience as a critical care nurse, you likely meet the experience requirement. However, it's always best to review the job description and requirements carefully to ensure you meet all the qualifications. Additionally, the job description mentions that Advanced Cardiac Life Support (ACLS) and CRRT (continuous renal replacement therapy) are assets, but not necessarily required. It's possible that having 5 years of experience as a critical care nurse may have provided you with the necessary skills and knowledge to perform these tasks. To confirm your eligibility, I would recommend reviewing the job description and requirements again, and then contacting the Human Resources department directly to inquire about the application process and any specific requirements for the position. |
3 | How is it like living in that city? | Nanaimo, located on central Vancouver Island, offers a unique blend of natural beauty, recreational opportunities, and a vibrant community. Here are some aspects of living in Nanaimo: ### Natural Beauty and Outdoor Activities - **Scenic Views**: Nanaimo is a harbour city with stunning ocean views and easy access to beaches. You can watch cruise ships pass by or enjoy the waterfront. - **Outdoor Activities**: The city is surrounded by hiking and biking trails, making it a great place for outdoor enthusiasts. You can also engage in water sports, fishing, and other recreational activities. ### Community and Lifestyle - **Close-Knit Community**: Nanaimo has a welcoming and close-knit community. The city offers a laid-back lifestyle with a focus on family and community events. - **Cultural Opportunities**: There are various cultural and recreational opportunities, including local markets, festivals, and arts events. ### Accessibility - **Convenient Location**: Nanaimo is a convenient access point to the mainland, making it easy to travel to other parts of British Columbia. The city is well-connected by ferry and air services. - **Proximity to Other Regions**: It's centrally located on Vancouver Island, providing easy access to other beautiful regions like Victoria, Tofino, and the Comox Valley. ### Economy and Employment - **Healthcare Sector**: As a healthcare professional, you'll be part of a significant sector in the region. Island Health is a major employer, providing numerous opportunities for career growth. - **Technology and Other Industries**: Nanaimo also has a growing technology sector and various other industries, contributing to a diverse economy. ### Education and Family Life - **Educational Institutions**: The city has good educational facilities, including schools and Vancouver Island University. - **Family-Friendly**: Nanaimo is considered a great place to raise a family, with numerous parks, recreational facilities, and community programs. ### Climate - **Mild Climate**: Nanaimo enjoys a mild climate with warm, dry summers and mild, wet winters. This makes it possible to enjoy outdoor activities year-round. ### Cost of Living - **Affordable Housing**: Compared to larger cities like Vancouver, Nanaimo offers more affordable housing options, making it an attractive place to live for many families and professionals. Overall, Nanaimo offers a balanced lifestyle with a mix of natural beauty, community spirit, and professional opportunities, making it an appealing place to live and work. | Nanaimo, located on central Vancouver Island, is a vibrant city with a rich cultural heritage. It's the largest population centre on the island and offers a variety of recreational and cultural opportunities. The city is known for its beautiful harbour and is a convenient access point to the mainland. It's also a gateway to the Nuu-chah-nulth Territory, with Port Alberni being a prized fishing and water sports destination. The city has a warm and welcoming community, and it's a great place to raise a family. There are also opportunities for outdoor activities, with easy access to hiking and biking trails. | Nanaimo is a city located on the east coast of Vancouver Island, British Columbia, Canada. It's a charming city with a population of around 90,000 people. Here are some aspects of living in Nanaimo: Pros: 1. **Natural beauty**: Nanaimo is surrounded by mountains, forests, and waterways, making it a great place for outdoor enthusiasts. The city has many parks, trails, and beaches to explore. 2. **Mild climate**: Nanaimo has a mild oceanic climate, with mild winters and cool summers. The average temperature ranges from 2°C to 22°C (36°F to 72°F) throughout the year. 3. **Affordable cost of living**: Compared to other cities in British Columbia, Nanaimo has a relatively low cost of living. Housing, food, and transportation costs are lower than in Vancouver or Victoria. 4. **Community-oriented**: Nanaimo has a strong sense of community, with many local events and festivals throughout the year. The city has a small-town feel, making it easy to get to know your neighbors and feel connected. Cons: 1. **Limited job opportunities**: While Nanaimo has a growing economy, job opportunities may be limited in certain industries. However, the city is investing in new developments and initiatives to attract businesses and talent. 2. **Traffic and congestion**: Nanaimo's roads can get congested, especially during rush hour. However, the city is working to improve traffic flow and public transportation options. 3. **Weather**: While the climate is mild, Nanaimo can experience rain and overcast skies, especially during the winter months. However, this is typical of the Pacific Northwest region. Overall, Nanaimo is a great place to live for those who enjoy outdoor activities, a relaxed pace of life, and a strong sense of community. |